# Scraping HelloWork

In [82]:
from dataclasses import dataclass
from datetime import datetime
import json
import time
from typing import List

import bs4
import numpy as np
import requests
from rich import print

In [75]:
def get_timestamp():
    """Help for file horodating"""
    return datetime.now().strftime("%Y-%m-%d-%H:%M:%S")

In [2]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)\
    AppleWebKit/537.36 (KHTML, like Gecko)\
        Chrome/116.0.0.0 Safari/537.36'
URL = 'https://www.hellowork.com/fr-fr/emplois/40663563.html?utm_source=adzuna&utm_medium=Metamoteurs-cpc&utm_campaign=classique%2B-%2Binformatique&utm_term=40663563&utm_content=fluxadzuna&utm_source=adzuna&utm_medium=adzuna'

In [22]:
resp = requests.get(URL, headers={'User-Agent': USER_AGENT})
html = bs4.BeautifulSoup(resp.text, 'html.parser')

In [4]:
resp.status_code

200

In [5]:
html


<!DOCTYPE html>

<!--
En visualisant le code source vous acceptez les CGU accessibles à l’adresse : https://www.hellowork.com/fr-fr/page/cgu.html
Notamment l’article 8.2
L’Utilisateur peut uniquement utiliser le Site aux fins d’accéder aux Services, ce exclusivement à des fins privées et non commerciales.
Il est notamment expressément reconnu et accepté par l’Utilisateur que l’utilisation de systèmes automatisés ou de logiciels pour extraire des données du Site, en particulier le Contenu et la CVthèque (« capture de données d’écran » / « screen scraping / web scraping» notamment), à quelques fins que ce soit, commerciales ou non, est strictement interdite, à moins que HelloWork n’ait conclu une convention de licence écrite autorisant expressément l’Utilisateur à extraire une partie des données du Site.
-->
<html class="tw-scroll-smooth" data-controller="fetch-account-data" data-fetch-account-data-url-value="/fr-fr/candidat/accountdata" lang="fr-fr">
<head>
<meta charset="utf-8"/>
<scr

In [18]:
type(html)

bs4.BeautifulSoup

# Sum-up

It's the section on the right of the page.

## 🔎 Search...

In [10]:
sections = html.find_all('section')
len(sections)

6

In [11]:
sections[-1]

<section class="tw-flex tw-flex-col lg:tw-p-6 lg:tw-max-w-[352px] lg:tw-border lg:tw-border-greyLighter tw-rounded-sm tw-bg-white tw-bg-opacity-30">
<h2 class="tw-flex lg:tw-hidden tw-text-lg sm:tw-text-2xl tw-border-b sm:tw-border-0 tw-border-greyLighter tw-pb-3 tw-mb-6 sm:tw-mb-0">
        En résumé :
    </h2>
<span class="tw-text-lg lg:tw-text-xl tw-line-clamp-2 tw-mb-3 lg:tw-mb-1">
        Analyste Fonctionnel H/F
    </span>
<div class="tw-inline-flex tw-flex-wrap">
<a class="tw-text-base tw-mr-2 tw-link-text tw-mb-2" href="/fr-fr/entreprises/externatic-45626.html" target="_blank" title="Externatic recrutement">
                    Externatic
                </a>
<div class="tw-typo-s-bold tw-mb-2 lg:tw-mb-3">
<span class="tw-text-candidate tw-flex tw-items-center">
<svg class="tw-mr-1 tw-w-4 tw-h-[1.125rem]" data-controller="cross-origin-svg" data-cross-origin-svg-url-value="https://f.hellowork.com/hellowork/tags/2.39.1//img/svg/illustrations/super-recruiter.svg">
</svg>
    Sup

In [15]:
results = sections[-1].find_all('li')
results

[<li class="tw-tag-contract-s tw-readonly">
             Angers - 49
         </li>,
 <li class="tw-tag-contract-s tw-readonly">
             CDI
         </li>,
 <li class="tw-tag-primary-s tw-readonly">
 <span>🏠</span> Télétravail partiel
             </li>,
 <li class="tw-block tw-tag-primary-s tw-readonly tw-w-fit tw-whitespace-nowrap tw-text-ellipsis tw-overflow-hidden">
                     Bac +5
                 </li>,
 <li class="tw-block tw-tag-primary-s tw-readonly tw-w-fit tw-whitespace-nowrap tw-text-ellipsis tw-overflow-hidden">
                     Services aux Entreprises
                 </li>,
 <li class="tw-block tw-tag-primary-s tw-readonly tw-w-fit tw-whitespace-nowrap tw-text-ellipsis tw-overflow-hidden">
                     Exp. - 1 an
                 </li>]

In [17]:
[result.text.strip() for result in results]

['Angers - 49',
 'CDI',
 '🏠 Télétravail partiel',
 'Bac +5',
 'Services aux Entreprises',
 'Exp. - 1 an']

## 🛠️ Function

In [40]:
def get_hw_sum_up(page: bs4.BeautifulSoup) -> List[str]:
    results = (page
               .find_all('section')
               [-1]
               .find_all('li')
              )
    return [result.text.strip()
            for result in results]

In [41]:
get_hw_sum_up(html)

['Angers - 49',
 'CDI',
 '🏠 Télétravail partiel',
 'Bac +5',
 'Services aux Entreprises',
 'Exp. - 1 an']

# Profile

## 🔎 Search

In [25]:
parags = html.find_all('p')
len(parags)

11

In [26]:
parags

[<p class="tw-menu-element-appears peer-parent-checked:tw-delay-[calc(var(--tw-burger-item-start)_+_4_*_var(--tw-burger-item-delay))] tw-mt-10 tw-mb-4 tw-uppercase tw-typo-xs-bold tw-pb-2 tw-border-b tw-border-greyLighter">
                         Télécharger l’app
                     </p>,
 <p class="tw-w-full tw-flex tw-items-center tw-justify-center">
 <svg class="tw-h-4 tw-w-4 tw-absolute tw-left-4" data-controller="cross-origin-svg" data-cross-origin-svg-url-value="https://f.hellowork.com/hellowork/tags/2.39.1//img/svg/nav/arrow-left.svg"></svg>
                             Emploi
                         </p>,
 <p class="tw-typo-long-m tw-mb-8 sm:tw-mb-12 tw-break-words">
                 Le Poste :<br/><br/>Au sein d'une équipe de 15 personnes, organisée en mode Agile, votre rôle consiste à :<br/><br/>* Réaliser les analyses détaillées des besoins clients<br/>* Animer les groupes de travail<br/>* Participer au maquettage des solutions fonctionnelles<br/>* Rédiger les spécifica

In [31]:
parags[3]

<p class="tw-typo-long-m tw-mb-8 sm:tw-mb-12 tw-break-words">
                Ce que vous apportez :<br/><br/>Hard skills<br/><br/>* première expérience fonctionnelle dans l'édition de logiciels (ERP, GTA, SIRH ou le développement d'applications métiers).<br/>* orienté produit<br/><br/>Soft skills<br/><br/>* organisation<br/>* autonomie<br/>* sens du service<br/><br/>À propos d'Externatic<br/><br/>Cabinet de recrutement Tech, la mission d'Externatic est de faciliter la rencontre entre candidats et entreprises. Nous mettons à votre disposition notre réseau et notre connaissance du marché de la Tech (étude des salaires, tendances).<br/><br/>Notre moteur : vous accompagner sur du long terme pour trouver l'opportunité en CDI, qui correspond à votre projet professionnel, et surtout vous proposer un accès privilégié à des opportunités cachées au sein de pépites (startup / éditeur / DSI / PME).<br/><br/>Chez nous, le côté humain prime et nous sommes transparents sur nos actions : ici, chaque 

In [32]:
parags[3].text

"\n                Ce que vous apportez :Hard skills* première expérience fonctionnelle dans l'édition de logiciels (ERP, GTA, SIRH ou le développement d'applications métiers).* orienté produitSoft skills* organisation* autonomie* sens du serviceÀ propos d'ExternaticCabinet de recrutement Tech, la mission d'Externatic est de faciliter la rencontre entre candidats et entreprises. Nous mettons à votre disposition notre réseau et notre connaissance du marché de la Tech (étude des salaires, tendances).Notre moteur : vous accompagner sur du long terme pour trouver l'opportunité en CDI, qui correspond à votre projet professionnel, et surtout vous proposer un accès privilégié à des opportunités cachées au sein de pépites (startup / éditeur / DSI / PME).Chez nous, le côté humain prime et nous sommes transparents sur nos actions : ici, chaque offre d'emploi correspond à un poste réel !Externatic en bref :* +700 postes ouverts HORS ESN* +94% de périodes d'essais validées* 30 consultants pour vou

❗The tag erasing completely breaks the structure of the paragraph:
- You don't keep the enumeration structure (with the `*`)
- Some words are therefore mixed.

<font color='orangered'>👉 **Maybe, for the moment, these could be kept as `soup` objects.**</font>

In [34]:
parags[3].text

"\n                Ce que vous apportez :Hard skills* première expérience fonctionnelle dans l'édition de logiciels (ERP, GTA, SIRH ou le développement d'applications métiers).* orienté produitSoft skills* organisation* autonomie* sens du serviceÀ propos d'ExternaticCabinet de recrutement Tech, la mission d'Externatic est de faciliter la rencontre entre candidats et entreprises. Nous mettons à votre disposition notre réseau et notre connaissance du marché de la Tech (étude des salaires, tendances).Notre moteur : vous accompagner sur du long terme pour trouver l'opportunité en CDI, qui correspond à votre projet professionnel, et surtout vous proposer un accès privilégié à des opportunités cachées au sein de pépites (startup / éditeur / DSI / PME).Chez nous, le côté humain prime et nous sommes transparents sur nos actions : ici, chaque offre d'emploi correspond à un poste réel !Externatic en bref :* +700 postes ouverts HORS ESN* +94% de périodes d'essais validées* 30 consultants pour vou

## 🛠️ Function

In [42]:
def get_hw_profile(page: bs4.BeautifulSoup) -> bs4.element.Tag:
    return page.find_all('p')[3]

In [43]:
get_hw_profile(html)

<p class="tw-typo-long-m tw-mb-8 sm:tw-mb-12 tw-break-words">
                Ce que vous apportez :<br/><br/>Hard skills<br/><br/>* première expérience fonctionnelle dans l'édition de logiciels (ERP, GTA, SIRH ou le développement d'applications métiers).<br/>* orienté produit<br/><br/>Soft skills<br/><br/>* organisation<br/>* autonomie<br/>* sens du service<br/><br/>À propos d'Externatic<br/><br/>Cabinet de recrutement Tech, la mission d'Externatic est de faciliter la rencontre entre candidats et entreprises. Nous mettons à votre disposition notre réseau et notre connaissance du marché de la Tech (étude des salaires, tendances).<br/><br/>Notre moteur : vous accompagner sur du long terme pour trouver l'opportunité en CDI, qui correspond à votre projet professionnel, et surtout vous proposer un accès privilégié à des opportunités cachées au sein de pépites (startup / éditeur / DSI / PME).<br/><br/>Chez nous, le côté humain prime et nous sommes transparents sur nos actions : ici, chaque 

# Tests

In [79]:
with open('../data/adzuna_url_jobs_2023-10-10-20:14:40.json', 'r') as read_file:
     urls = [(item['id'], item['hellowork_url'])
             for item in json.load(read_file)]

In [80]:
urls

[('4354266725',
  'https://www.hellowork.com/fr-fr/emplois/40972526.html?utm_source=adzuna&utm_medium=Metamoteurs-cpc&utm_campaign=classique%2B-%2Binformatique&utm_term=40972526&utm_content=fluxadzuna&utm_source=adzuna&utm_medium=adzuna'),
 ('4332382286',
  'https://www.free-work.com/tech-it/administrateur-bdd/job-mission/pot7527-un-administrateur-bd-sur-saint-denis-la-plaine?utm_source=adzuna&utm_medium=aggregator&utm_campaign=adzuna_aggregator_fr_all_fl'),
 ('4333958749',
  'https://www.free-work.com/tech-it/developpeur-php/job-mission/developpement-fullstack-laravel-reactjs-ref-kb?utm_source=adzuna&utm_medium=aggregator&utm_campaign=adzuna_aggregator_fr_all_fl'),
 ('4338499486',
  'https://www.hellowork.com/fr-fr/emplois/40714731.html?utm_source=adzuna&utm_medium=Metamoteurs-cpc&utm_campaign=classique%2B-%2Binformatique&utm_term=40714731&utm_content=fluxadzuna&utm_source=adzuna&utm_medium=adzuna'),
 ('4309544618', None),
 ('4316187992', None),
 ('4356050484', None),
 ('4314113406',


In [83]:
sum_ups = []

for idx, (id, url) in enumerate(urls, 1):
    if url is not None:
        resp = requests.get(url, headers={'User-Agent': USER_AGENT})
        html = bs4.BeautifulSoup(resp.text, 'html.parser')
        sum_ups.append((id, url, get_hw_sum_up(html)))
        pause = 3 + 4 * np.random.rand()
        time.sleep(pause)
    else:
        sum_ups.append((id, None))

In [84]:
sum_ups

[('4354266725',
  'https://www.hellowork.com/fr-fr/emplois/40972526.html?utm_source=adzuna&utm_medium=Metamoteurs-cpc&utm_campaign=classique%2B-%2Binformatique&utm_term=40972526&utm_content=fluxadzuna&utm_source=adzuna&utm_medium=adzuna',
  ['Saint-Gérand - 56',
   'CDI',
   'Bac +3, Bac +4',
   'Bac +5',
   'Industrie Agro-alimentaire',
   'Exp. 1 à 7 ans',
   'Exp. + 7 ans']),
 ('4332382286',
  'https://www.free-work.com/tech-it/administrateur-bdd/job-mission/pot7527-un-administrateur-bd-sur-saint-denis-la-plaine?utm_source=adzuna&utm_medium=aggregator&utm_campaign=adzuna_aggregator_fr_all_fl',
  []),
 ('4333958749',
  'https://www.free-work.com/tech-it/developpeur-php/job-mission/developpement-fullstack-laravel-reactjs-ref-kb?utm_source=adzuna&utm_medium=aggregator&utm_campaign=adzuna_aggregator_fr_all_fl',
  []),
 ('4338499486',
  'https://www.hellowork.com/fr-fr/emplois/40714731.html?utm_source=adzuna&utm_medium=Metamoteurs-cpc&utm_campaign=classique%2B-%2Binformatique&utm_term=40

In [85]:
# Dumping results
ts = get_timestamp()
dump_path = f'../data/adzuna_hw_scraping_{ts}.json'
with open(dump_path, 'w', encoding='utf-8') as dump_file:
    json.dump(sum_ups, dump_file, indent=4, ensure_ascii=False) 
print(f'\t[cyan]{dump_path} dumped![/cyan]')  

../data/adzuna_hw_scraping_2023-10-11-08:23:16.json dumped!